In [1]:
import numba_dppy as dppy
import dpctl
import numpy as np


gpu = dpctl.select_gpu_device()
print(gpu.name)

Intel(R) UHD Graphics P630 [0x3e96]


In [2]:
rng = np.random.RandomState(42)
data = rng.normal(size=(1_000_000, 100)).astype(np.float32)
cluster_indices = np.zeros_like(data, dtype=np.int32)
centroids = data[rng.choice(np.arange(data.shape[0]), 100)]
print(f"data size: {data.nbytes / 1e6} MB")

data size: 400.0 MB


In [3]:
# The following way to device allocate the USM arrays is currently not
# supported: it would make the numba_dppy kernel compiler raise a
# NotImplementedError.

# import dpctl.tensor as dpt
# 
# def convert_to_usm(data, buffer="shared"):
#     data_usm = dpt.usm_ndarray(data.shape, dtype=data.dtype, buffer=buffer)
#     data_usm.usm_data.copy_from_host(data.ravel().view("u1"))
#     return data_usm

In [4]:
import dpctl.memory as dpmem


def convert_to_usm(data, buffer="shared"):
    data_usm = dpmem.MemoryUSMShared(data.nbytes)
    data_usm.copy_from_host(data.ravel().view("u1"))
    return np.ndarray(data.shape, buffer=data_usm, dtype=data.dtype)


with dpctl.device_context(gpu):
    data_usm = convert_to_usm(data)
    centroids_usm = convert_to_usm(centroids)
    cluster_indices_usm = convert_to_usm(cluster_indices)

In [5]:
assert data_usm.base.sycl_device == gpu

## Numba DPPY / GPU implementation of the main k-means kernel

In [6]:
def make_dppy_kernel(n_samples, n_features, n_centroids):
    @dppy.kernel
    def assign(data, cluster_indices, centroids):
        sample_idx = dppy.get_global_id(0)
        if sample_idx < n_samples:
            min_dist = -1
            for centroid_idx in range(n_centroids):
                dist = 0.0
                for feature_idx in range(n_features):
                    dist += (
                        data[sample_idx, feature_idx]
                        - centroids[centroid_idx, feature_idx]
                    ) ** 2
                if min_dist > dist or min_dist < 0:
                    min_dist = dist
                    cluster_indices[sample_idx] = centroid_idx

    return assign


assign_dppy_gpu = make_dppy_kernel(data_usm.shape[0], data_usm.shape[1], centroids_usm.shape[0])

In [7]:
from time import perf_counter


def timeit(func, *args, n_repeats=5, verbose=True, **kwargs):
    times = []
    for _ in range(n_repeats):
        tic = perf_counter()
        func(*args, **kwargs)
        times.append(perf_counter() - tic)
        if verbose:
            print(".", end="", flush=True)
    mean_time = np.mean(times)
    if verbose:
        print(f"\naverage per call duration: {np.mean(times):.3f}s")
    return mean_time

In [8]:
with dpctl.device_context(gpu):
    assign_dppy_gpu_time = timeit(
        assign_dppy_gpu[data.shape[0], dppy.DEFAULT_LOCAL_SIZE],
        data_usm,
        cluster_indices_usm,
        centroids_usm,
    )

.....
average per call duration: 1.317s


In [9]:
cluster_indices_usm

array([[46, 46, 46, ..., 46, 46, 46],
       [56, 56, 56, ..., 56, 56, 56],
       [ 1,  1,  1, ...,  1,  1,  1],
       ...,
       [76, 76, 76, ..., 76, 76, 76],
       [38, 38, 38, ..., 38, 38, 38],
       [56, 56, 56, ..., 56, 56, 56]], dtype=int32)

In [10]:
assert cluster_indices_usm.base.sycl_device == gpu

## Numba DPPY / CPU implementation of the main k-means kernel (broken)

In [11]:
cpu = dpctl.select_cpu_device()
cpu.name

'Intel(R) Xeon(R) E-2176G CPU @ 3.70GHz'

In [12]:
with dpctl.device_context(cpu):
    data_usm_cpu = convert_to_usm(data)
    centroids_usm_cpu = convert_to_usm(centroids)
    cluster_indices_usm_cpu = convert_to_usm(cluster_indices)

    assign_dppy_cpu = make_dppy_kernel(
        data_usm_cpu.shape[0],
        centroids_usm_cpu.shape[1],
        cluster_indices_usm_cpu.shape[0],
    )

In [13]:
# The following does not work and crashes the jupyter kernel:

# with dpctl.device_context(cpu):
#     assign_dppy_cpu_time = timeit(
#         assign_dppy_cpu[data.shape[0], dppy.DEFAULT_LOCAL_SIZE],
#         data_usm_cpu,
#         centroids_usm_cpu,
#         cluster_indices_usm_cpu,
#     )

## Numba CPU implementation of the main k-means kernel

In [14]:
import numba


# numba.config.THREADING_LAYER = 'omp'  # similar speed as 'tbb'


def make_numba_kernel(n_samples, n_features, n_centroids):
    @numba.njit(parallel=True)
    def assign(data, cluster_indices, centroids):
        for sample_idx in numba.prange(n_samples):
            min_dist = -1
            for centroid_idx in range(n_centroids):
                dist = 0.0
                for feature_idx in range(n_features):
                    dist += (
                        data[sample_idx, feature_idx]
                        - centroids[centroid_idx, feature_idx]
                    ) ** 2
                if min_dist > dist or min_dist < 0:
                    min_dist = dist
                    cluster_indices[sample_idx] = centroid_idx

    return assign

assign_numba = make_numba_kernel(data.shape[0], data.shape[1], centroids.shape[0])

In [15]:
assign_numba_cpu_time = timeit(assign_numba, data, cluster_indices, centroids)

.....
average per call duration: 2.811s


In [16]:
np.testing.assert_array_equal(cluster_indices, cluster_indices_usm)

In [17]:
print(f"Threading: {numba.threading_layer()} with {numba.get_num_threads()} threads")

Threading: tbb with 12 threads


In [18]:
print(
    f"dppy-GPU vs numba-CPU speed-up: "
    f"{assign_numba_cpu_time / assign_dppy_gpu_time:.1f}x"
)

dppy-GPU vs numba-CPU speed-up: 2.1x
